In [4]:
import torch

x = torch.randn(2, 3)
mask = torch.tensor([True, False, True])  # shape (3,)
x.masked_fill(mask, 0)  # ❌ 错，因为 (3,) 无法广播到 (2, 3)

a = torch.rand((3, 5))
print(a)
print(a.unsqueeze(1))
print(a.unsqueeze(1).unsqueeze(-1))

tensor([[0.3969, 0.2800, 0.8349, 0.2901, 0.6935],
        [0.8142, 0.6477, 0.4673, 0.2253, 0.5960],
        [0.6400, 0.8461, 0.3737, 0.8131, 0.2501]])
tensor([[[0.3969, 0.2800, 0.8349, 0.2901, 0.6935]],

        [[0.8142, 0.6477, 0.4673, 0.2253, 0.5960]],

        [[0.6400, 0.8461, 0.3737, 0.8131, 0.2501]]])
tensor([[[[0.3969],
          [0.2800],
          [0.8349],
          [0.2901],
          [0.6935]]],


        [[[0.8142],
          [0.6477],
          [0.4673],
          [0.2253],
          [0.5960]]],


        [[[0.6400],
          [0.8461],
          [0.3737],
          [0.8131],
          [0.2501]]]])


In [11]:
import torch



N, num_head, q_seq_len, k_seq_len = 1, 2, 5, 5
mul_res = torch.tensor([[
[
    [1.2, 0.3, 0.4, 0.2, 0.1],  # token 0
    [0.3, 1.0, 0.6, 0.5, 0.1],  # token 1
    [0.4, 0.6, 1.3, 0.7, 0.2],  # token 2
    [0.2, 0.5, 0.7, 1.1, 0.4],  # token 3
    [0.1, 0.1, 0.2, 0.4, 1.5]   # token 4 (padding)
],
[
    [0.4, 0.6, 1.3, 0.7, 0.2],  
    [0.2, 0.5, 0.7, 1.1, 0.4],  
    [0.1, 0.1, 0.2, 0.4, 1.5],
    [1.2, 0.3, 0.4, 0.2, 0.1],  
    [0.3, 1.0, 0.6, 0.5, 0.1],  
]
]])
# print(mul_res.shape)    # (N, num_head, q_len, k_len)
key_padding_mask = torch.tensor([[False, False, False, False, True]])  # 只有最后一个 token 是 padding
# print(key_padding_mask)
attn_mask = torch.triu(torch.ones((q_seq_len, k_seq_len)), diagonal=1).bool()   # (q_len=5, k_len=5)

if key_padding_mask is not None:
    key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
    key_padding_mask = key_padding_mask.expand((N, num_head, q_seq_len, k_seq_len))
    mul_res = torch.masked_fill(mul_res, key_padding_mask, -torch.inf)
print(mul_res.shape)
print(mul_res)
if attn_mask is not None:
    attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)
    attn_mask = attn_mask.expand((N, num_head, q_seq_len, k_seq_len))
    mul_res = torch.masked_fill(mul_res, attn_mask, -torch.inf)
print(mul_res.shape)
print(mul_res)

# key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
# key_padding_mask = key_padding_mask.expand((1, num_head, 5, 5))
# print(key_padding_mask)
# print(torch.softmax(torch.tensor([float("inf"), float("inf")]), dim=0))
# attn_output_weights = attn_output_weights.masked_fill(, float('-inf'))
# print(attn_output_weights)

torch.Size([1, 2, 5, 5])
tensor([[[[1.2000, 0.3000, 0.4000, 0.2000,   -inf],
          [0.3000, 1.0000, 0.6000, 0.5000,   -inf],
          [0.4000, 0.6000, 1.3000, 0.7000,   -inf],
          [0.2000, 0.5000, 0.7000, 1.1000,   -inf],
          [0.1000, 0.1000, 0.2000, 0.4000,   -inf]],

         [[0.4000, 0.6000, 1.3000, 0.7000,   -inf],
          [0.2000, 0.5000, 0.7000, 1.1000,   -inf],
          [0.1000, 0.1000, 0.2000, 0.4000,   -inf],
          [1.2000, 0.3000, 0.4000, 0.2000,   -inf],
          [0.3000, 1.0000, 0.6000, 0.5000,   -inf]]]])
torch.Size([1, 2, 5, 5])
tensor([[[[1.2000,   -inf,   -inf,   -inf,   -inf],
          [0.3000, 1.0000,   -inf,   -inf,   -inf],
          [0.4000, 0.6000, 1.3000,   -inf,   -inf],
          [0.2000, 0.5000, 0.7000, 1.1000,   -inf],
          [0.1000, 0.1000, 0.2000, 0.4000,   -inf]],

         [[0.4000,   -inf,   -inf,   -inf,   -inf],
          [0.2000, 0.5000,   -inf,   -inf,   -inf],
          [0.1000, 0.1000, 0.2000,   -inf,   -inf],
       

In [4]:
from collections import OrderedDict
from transformers import AutoModel

ckp = "google-bert/bert-base-uncased"

bert = AutoModel.from_pretrained(ckp)
print(bert)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [7]:
import torch
import torch.nn as nn


emb = nn.Embedding(10, 5)

print(emb(torch.tensor([[1, 3], [1, 2]])))

tensor([[[-0.3060,  1.7448,  0.5608, -1.4517,  0.3484],
         [-1.2369, -0.2216,  0.1535, -0.0511,  0.7589]],

        [[-0.3060,  1.7448,  0.5608, -1.4517,  0.3484],
         [-0.2811, -1.6255, -1.2007, -0.1995,  0.1456]]],
       grad_fn=<EmbeddingBackward0>)
